## Modules à importer

In [1]:
import math
import scipy
import pickle
import numpy as np
import pandas as pd
import iminuit as im
import ipywidgets as ipw
#from variaIa import tools_ar
from variaIa import tools_ar_sc
from importlib import reload
from variaIa import stretchevol
#from variaIa.tools_ar import surveys, df_cons, df_nc, df_c, df_full, z_max
from variaIa.tools_ar_sc import surveys, df_cons, df_nc, df_c, df_full, z_max
from IPython.display import display, Latex
import matplotlib.pyplot as plt
import random

## Définitions pandas, surveys et sélection de l'échantillon à utiliser

### Définition des données à montrer

In [3]:
def set_survey(cons, su):
    global df
    if su == 'All':
        df = df_cons(cons)
    elif 'n' in su:
        df = df_cons(cons)[df_cons(cons)['survey'] != su.split('n')[-1]]
    else:
        df = df_cons(cons)[df_cons(cons)['survey'] == su]
    return df

#int_set_su = ipw.interactive(set_survey, cons=tools_ar.cons, su=tools_ar.su)
int_set_su = ipw.interactive(set_survey, cons=tools_ar_sc.cons, su=tools_ar_sc.su)
display(int_set_su)

smap = plt.cm.get_cmap('cividis')
colors = {'SNF': plt.cm.binary(0.9),
          'SDSS': smap(0.1),
          'PS1': smap(0.5),
          'SNLS': smap(0.8),
          'HST': smap(0.99),  # for cividis
          #'HST': smap(300)}  # for viridis
          'All': (255/255, 51/255, 0/255)} 

def alphacolors(alpha, survey):
    return(colors[survey][0], colors[survey][1], colors[survey][2], alpha)

df_nHST = df_nc[df_nc['survey'] != 'HST']
surveysnl = surveys[0:1]+surveys[2:]

interactive(children=(Checkbox(value=False, description='Super conservative'), Dropdown(description='Survey:',…

## Get proba

In [4]:
def get_proba(model):
    return np.exp((evol3G2M2S.get_aic() - model.get_aic())/2)

## Minimisation des modèles à mettre dans le _pandas_

In [8]:
########################################### Evol Simple

evol1G1M1S = stretchevol.Evol1G1M1S(df)

print('1G1M1S')

evol1G1M1S.minimize()
#evol1G1M1S.show_model_tot()

########################################### Evol Kessler

evol1G1M2S = stretchevol.Evol1G1M2S(df)

print('1G1M2S')

evol1G1M2S.minimize()
#evol1G1M2S.show_model_tot()

########################################### Evol Howell

evol2G2M2S = stretchevol.Evol2G2M2S(df)

print('2G2M2S')

evol2G2M2S.minimize()
#evol2G2M2S.scatter()

########################################### Evol Howell Fixed

evol2G2M2SF = stretchevol.Evol2G2M2SF(df)

print('2G2M2SF')

evol2G2M2SF.minimize()
#evol2G2M2SF.scatter()

########################################### Evol Howell Z

evol2G2M2Sz = stretchevol.Evol2G2M2S(df, py=False)

print('2G2M2Sz')

evol2G2M2Sz.minimize()
#evol2G2M2S.scatter()

########################################### Evol Howell Fixed Z

evol2G2M2SFz = stretchevol.Evol2G2M2SF(df, py=False)

print('2G2M2SFz')

evol2G2M2SFz.minimize()
#evol2G2M2SF.scatter()

########################################### Evol NR 2S

evol3G2M2S = stretchevol.Evol3G2M2S(df)

print('3G2M2S')

evol3G2M2S.minimize()
#evol3G2M2S.scatter()

########################################### Evol NR 2S z

evol3G2M2Sz = stretchevol.Evol3G2M2S(df, py=False)

print('3G2M2Sz')

evol3G2M2Sz.minimize()
#evol3G2M2S.scatter()

########################################### Evol NR 2S Fixed

evol3G2M2SF = stretchevol.Evol3G2M2SF(df)

print('3G2M2SF')

evol3G2M2SF.minimize(limit_f = (0,1))
#evol3G2M2SF.scatter()

########################################### Evol NR 2S Fixed Z

evol3G2M2SFz = stretchevol.Evol3G2M2SF(df, py=False)

print('3G2M2SFz')

evol3G2M2SFz.minimize(limit_f = (0,1))
#evol3G2M2SF.scatter()

########################################### Evol NR 3S

evol3G3M3S = stretchevol.Evol3G3M3S(df)

print('3G3M3S')

evol3G3M3S.minimize()

########################################### Evol NR 3S Z

evol3G3M3Sz = stretchevol.Evol3G3M3S(df, py=False)

print('3G3M3Sz')

evol3G3M3Sz.minimize()

########################################### Per Sample Asym
gen = stretchevol.generic()
gen.set_model('Evol1G1M2S')

assymSNF = gen.fit(set_survey(tools_ar_sc.cons.value, 'SNF'))

assymPS1 = gen.fit(set_survey(tools_ar_sc.cons.value, 'PS1'))

assymSDSS = gen.fit(set_survey(tools_ar_sc.cons.value, 'SDSS'))

assymSNLS = gen.fit(set_survey(tools_ar_sc.cons.value, 'SNLS'))

assymHST = gen.fit(set_survey(tools_ar_sc.cons.value, 'HST'))

assymTOT = gen.fit(set_survey(tools_ar_sc.cons.value, 'All'))

assym = {'SNF': assymSNF,
         'SDSS': assymSDSS,
         'PS1': assymPS1,
         'SNLS': assymSNLS,
         'HST': assymHST}

print('Per Sample')

1G1M1S
1G1M2S
2G2M2S
2G2M2SF
2G2M2Sz
2G2M2SFz
3G2M2S
3G2M2Sz
3G2M2SF
3G2M2SFz
3G3M3S
3G3M3Sz
Per Sample


## Comparison dataframes

In [9]:
modèles = [evol3G2M2S, evol2G2M2S,
           evol1G1M2S, evol2G2M2SF, evol3G2M2SF,
           evol1G1M1S]

persample = [assym[k] for k in assym.keys()]

d_mod_comp = pd.DataFrame({'Name': [k.__doc__ for k in modèles[:-1]] + ['Per sample Asym'] + [k.__doc__ for k in modèles[-1:]],
                           'Fixed': [k.FIXED for k in modèles[:-1]] + ['True'] + [k.FIXED for k in modèles[-1:]],
                           'Free param': [len(k.FREEPARAMETERS) for k in modèles[:-1]] + ['3x5'] + [len(k.FREEPARAMETERS) for k in modèles[-1:]],
                           '$\mathcal{L}$': [round(k.get_logl(),1) for k in modèles[:-1]] + [round(np.sum([k.get_logl() for k in persample]),1)] + [round(k.get_logl(),1) for k in modèles[-1:]],
                           'AIC': [round(k.get_aic(),1) for k in modèles[:-1]] + [round(np.sum([k.get_aic() for k in persample]),1)] + [round(k.get_aic(),1) for k in modèles[-1:]],
                           '$\Delta$ AIC': [round(evol3G2M2S.get_aic()-k.get_aic(),1) for k in modèles[:-1]] + [round(evol3G2M2S.get_aic()-np.sum([k.get_aic() for k in persample]),1)] + [round(evol3G2M2S.get_aic()-k.get_aic(),1) for k in modèles[-1:]],
                           'Proba': [f'{get_proba(k):.1e}' for k in modèles[:-1]] + [f'{np.exp((evol3G2M2S.get_aic()-np.sum([k.get_aic() for k in persample]))/2):.1e}'] + [f'{get_proba(k):.1e}' for k in modèles[-1:]]
                          })

path = '../../../Data/tables/main_comp/df_comparaison'
#if tools_ar.cons.value:
#    path += '_cons'
if tools_ar_sc.cons.value:
    path += '_cons-supercut'
path += '.dat'
d_mod_comp.to_csv(path)

d_mod_comp

,Name,Fixed,Free param,$\mathcal{L}$,AIC,$\Delta$ AIC,Proba
0,Base,False,5,615.1,625.1,0.0,1.0e+00
1,Howell+drift,False,4,623.8,631.8,-6.8,3.4e-02
2,Asymmetric,True,3,643.9,649.9,-24.8,4.0e-06
3,Howell+const,True,5,639.0,649.0,-23.9,6.5e-06
4,Base+const,True,6,639.0,651.0,-25.9,2.4e-06
5,Per sample Asym,True,3x5,639.0,669.0,-43.9,3.0e-10
6,Gaussian,True,2,657.7,661.7,-36.7,1.1e-08


In [68]:
def get_proba(model):
    return np.exp((evol3G2M2Sz.get_aic() - model.get_aic())/2)

In [8]:
modèles = [evol2G2M2Sz, evol3G2M2Sz,
           evol1G1M2S, evol2G2M2SFz, evol3G2M2SFz,
           evol1G1M1S]

d_mod_comp = pd.DataFrame({'Name': [k.__doc__ for k in modèles[:-1]] + ['Per sample Asym'] + [k.__doc__ for k in modèles[-1:]],
                           'Fixed': [k.FIXED for k in modèles[:-1]] + ['True'] + [k.FIXED for k in modèles[-1:]],
                           'Free param': [len(k.FREEPARAMETERS) for k in modèles[:-1]] + ['3x5'] + [len(k.FREEPARAMETERS) for k in modèles[-1:]],
                           '$\mathcal{L}$': [round(k.get_logl(),1) for k in modèles[:-1]] + [1083.6 if cons.value else 1468.2] + [round(k.get_logl(),1) for k in modèles[-1:]],
                           'AIC': [round(k.get_aic(),1) for k in modèles[:-1]] + [1113.6 if cons.value else 1498.2] + [round(k.get_aic(),1) for k in modèles[-1:]],
                           '$\Delta$ AIC': [round(evol2G2M2Sz.get_aic()-k.get_aic(),1) for k in modèles[:-1]] + [-5.2 if cons.value else -10.9] + [round(evol2G2M2Sz.get_aic()-k.get_aic(),1) for k in modèles[-1:]],
                           'Proba': [get_proba(k) for k in modèles[:-1]] + [5.7e-6 if cons.value else 1.5e-7] + [get_proba(k) for k in modèles[-1:]]
                          })

path = '../../../Data/tables/main_comp/df_comparaison_zonly'
if cons.value:
    path += '_cons-supercut'
path += '.dat'
d_mod_comp.to_csv(path)

d_mod_comp

,Name,Fixed,Free param,$\mathcal{L}$,AIC,$\Delta$ AIC,Proba
0,Howell+drift,False,4,641.4,649.4,0.0,5.141495e-06
1,Base,False,5,637.5,647.5,1.9,1.339992e-05
2,Asymmetric,True,3,643.9,649.9,-0.5,4.029433e-06
3,Howell+const,True,5,639.0,649.0,0.5,6.471773e-06
4,Base+const,True,6,639.0,651.0,-1.5,2.380832e-06
5,Per sample Asym,True,3x5,1083.6,1113.6,-5.2,5.700000e-06
6,Gaussian,True,2,657.7,661.7,-12.3,1.099283e-08


## Get random samples the size of the conservative one

In [41]:
indexes = {survey: list(df_nc[df_nc['survey'] == survey].index)
           for survey in surveysnl[1:-1]}
todrop_len = {survey: len(df_nc[df_nc['survey'] == survey]) - \
                      len(df_c[df_c['survey'] == survey])
             for survey in surveysnl[1:-1]}

In [54]:
df_c_list = []
for i in range(200):
    todrop_dict = {survey: random.sample(indexes[survey],
                                         todrop_len[survey])
                   for survey in surveysnl[1:-1]}
    todrop = list(todrop_dict.values())[0] + \
             list(todrop_dict.values())[1] + \
             list(todrop_dict.values())[2]
    df_c_list.append(df_nc.drop(todrop))

In [86]:
pickle.dump(df_c_list, open('../../../Data/sne/df_redu', 'wb'))

## Minimize models using these samples

In [79]:
models_name = ['Evol3G2M2S', 'Evol2G2M2S', 'Evol1G1M2S', 'Evol3G2M2SF']
models_list = {name: [] for name in models_name}
models_AIC = {name: [] for name in models_name}
gen = stretchevol.generic()
for k in range(4):
    gen.set_model(models_name[k])
    for i in range(200):
        models_list[models_name[k]].append(gen.fit(df_c_list[i]))

for k in range(4):
    for i in range(200):
        models_AIC[models_name[k]].append(models_list[models_name[0]][i].get_aic() -
                                          models_list[models_name[k]][i].get_aic())

/home/nicolas/.installs/anaconda3/lib/python3.7/site-packages/variaIa-0.8.4-py3.7.egg/variaIa/stretchevol.py:749: RuntimeWarning: invalid value encountered in log
  mu_2, sigma_2)))


## Comparison dataframes

### Be sure of get_proba

In [69]:
def get_proba(model):
    return np.exp((evol3G2M2S.get_aic() - model.get_aic())/2)

### 4 models true one

In [77]:
modèles = [evol3G2M2S, evol2G2M2S,
           evol1G1M2S, evol3G2M2SF]

d_mod_comp = pd.DataFrame({'Name': [k.__doc__ for k in modèles], 
                           'Fixed': [k.FIXED for k in modèles],
                           'Free param': [len(k.FREEPARAMETERS) for k in modèles],
                           '$\mathcal{L}$': [round(k.get_logl(),1) for k in modèles],
                           'AIC': [round(k.get_aic(),1) for k in modèles],
                           '$\Delta$ AIC': [round(evol3G2M2S.get_aic()-k.get_aic(),1) for k in modèles],
                           'Proba': [get_proba(k) for k in modèles],
                          })
 
path = '../../../Data/tables/main_comp/df_comparaison_4mod'
if tools.cons.value:
    path += '_cons'
path += '.dat'
#d_mod_comp.to_csv(path)

#### Save 4 models tests ones

In [83]:
for i in range(200):
    modèles = [models_list[k][i] for k in models_name]
    d_mod_comp = pd.DataFrame({'Name': [k.__doc__ for k in modèles],
                               'Fixed': [k.FIXED for k in modèles],
                               'Free param': [len(k.FREEPARAMETERS) for k in modèles],
                               '$\mathcal{L}$': [round(k.get_logl(),1) for k in modèles],
                               'AIC': [round(k.get_aic(),1) for k in modèles],
                               '$\Delta$ AIC': [round(modèles[0].get_aic()-k.get_aic(),1) for k in modèles],
                               'Proba': [get_proba(k) for k in modèles],
                              })

    path = f'../../../Data/tables/main_comp/df_comparaison_4mod_{i}.dat'
    d_mod_comp.to_csv(path)

In [84]:
d_mod_comp

,Name,Fixed,Free param,$\mathcal{L}$,AIC,$\Delta$ AIC,Proba
0,Base,False,5,1080.9,1090.9,0.0,4.971085e-01
1,Howell+drift,False,4,1087.5,1095.5,-4.6,5.019025e-02
2,Asymmetric,True,3,1108.9,1114.9,-24.0,3.053724e-06
3,Base+const,True,6,1108.8,1120.8,-29.9,1.637083e-07
